### load ECCO model output for SASSIE and make basic plots

k.drushka // feb 2022

data from: https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Plotting_Tiles.html

download data by mounting ECCO Drive locally (Linux) and then copying files (e.g., May 01-09, 2011): 
```
sudo mount.davfs https://ecco.jpl.nasa.gov/drive/files /mnt/ecco_drive/
cd /mnt/ecco_drive/ECCO2/SASSIE/2011_demo/
for d in * do
cp -u $d/*_2011-05-0*.nc PATH-TO-YOUR-DATA-DRIVE/model/$d/
done
```


some code to plot ECCO data from: https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Plotting_Tiles.html

In [ ]:
## Imports 

# Native packages
from math import radians, degrees, sin, cos, asin, acos, sqrt
import datetime
import time
import sys
import os
import glob as glob


import numpy as np
import pandas as pd
import xarray as xr

# from xgcm import Grid
# import xgcm.grid

# data visualizations
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
import matplotlib.ticker as mticker


In [ ]:
# local directory where data are stored
basedir = '/data1/sassie/model/2011_demo/'

In [ ]:
# load files with xarray:

# specify which var to open:
thisvar = 'SALT'
files = glob.glob(basedir + '/' + thisvar + '*_day_mean/*nc') 
print(files)                
          

In [ ]:
# load files
# throws an error of decode_cf isn't specified 
ds = xr.open_mfdataset(files, combine='nested', concat_dim='time', decode_cf=False) 
# ds = xr.open_mfdataset(files, combine='nested', concat_dim='time') 
ds

In [ ]:
# XC, YC, Z are the same for each time, so redefine them
ds['XC'] = ds['XC'].isel(time=0)
ds['YC'] = ds['YC'].isel(time=0)
ds['Z'] = ds['Z'].isel(time=0)
ds['Z_bot'] = ds['Z_bot'].isel(time=0)
ds['Z_top'] = ds['Z_top'].isel(time=0)

In [ ]:
# simple plot
ds.SALT.isel(k=0, time=0, i=range(0,500), j=range(0,500)).plot(vmin=20, vmax=35)


In [ ]:
# limit to the beaufort - this is crude and imprecise, but works
irange = slice(0, 500)
jrange = slice(0, 500)
ds = ds.sel(i=irange, j=jrange)
ds

In [ ]:
# new plot to check the domain
ds.SALT.isel(k=0, time=0).plot(vmin=20, vmax=35)

In [ ]:
# nice plot using scatter

# index of time and depth to plot
ti = 0 
zi = 0

fig, ax1 = plt.subplots(1,1, 
        subplot_kw={'projection': ccrs.NorthPolarStereo(central_longitude=-150)}, 
        figsize=(6,6))

pc = ax1.scatter(ds.XC.data, 
                 ds.YC.data, 
                 c=ds[thisvar].isel(k=zi, time=ti).data,
                   transform=ccrs.PlateCarree(),
                   vmin = 25,
                   vmax = 35)

#  ----- map stuff
ax1.coastlines(color='none')  # coastline
ax1.set_extent([-170, -130,68, 80], crs=ccrs.PlateCarree())
gl = ax1.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3)
# ax1.set_title()
plt.colorbar(pc, ax=ax1, orientation="horizontal", pad=0.05).set_label(thisvar)
# land color
ax1.add_feature(cartopy.feature.LAND , facecolor=(.7,.7,.7))
# ticks
gl.ylocator = mticker.FixedLocator([68, 70, 72, 74, 76, 78])
gl.xlocator = mticker.FixedLocator([-170, -160, -150, -140, -130])
gl.top_labels = False
gl.bottom_labels = True

In [ ]:
# or, plot with pcolor (much faster)
lons = ds.XC
lats = ds.YC
# index of time and depth to plot
ti = 0 
zi = 0
# mask zero data (assumed to be land) with nan
data_to_plot = ds[thisvar].isel(k=zi, time=ti).where(ds.SALT.isel(k=zi, time=ti) !=0,np.nan)

fig, ax1 = plt.subplots(1,1, 
        subplot_kw={'projection': ccrs.NorthPolarStereo(central_longitude=-150)}, 
        figsize=(6,6))

plt.pcolormesh(lons, lats, data_to_plot,
               transform=ccrs.PlateCarree(),
              vmin=25, vmax=35);
plt.colorbar()

#  ----- map stuff
ax1.coastlines(color='none')  # coastline
ax1.set_extent([-170, -130,68, 80], crs=ccrs.PlateCarree())
gl = ax1.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3)
# ax1.set_title()
plt.colorbar(pc, ax=ax1, orientation="horizontal", pad=0.05).set_label(thisvar)
# land color
ax1.add_feature(cartopy.feature.LAND , facecolor=(.7,.7,.7))
# ticks
gl.ylocator = mticker.FixedLocator([68, 70, 72, 74, 76, 78])
gl.xlocator = mticker.FixedLocator([-170, -160, -150, -140, -130])
gl.top_labels = False
gl.bottom_labels = True

